# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [10]:
# Imports here
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from torch import optim
from torchvision import datasets, transforms, models
import wget
import time
from collections import OrderedDict
from PIL import Image

In [5]:
# importing the "tarfile" module
import tarfile

# open file
file = tarfile.open('flower_data.tar.gz')

# extracting file
file.extractall('./flowers')

file.close()

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [6]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [13]:
# TODO: Define your transforms for the training, validation, and testing sets
std = [0.229, 0.224, 0.225]
means = [0.485, 0.456, 0.406]
data_transforms = {
    'train': transforms.Compose([
    transforms.RandomRotation(30),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(means,std)]),
    'test': transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(means,std)]),
    'valid': transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(means,std)
    ]),
}

# TODO: Load the datasets with ImageFolder
image_datasets = {
    'train_data': datasets.ImageFolder(train_dir,transform = data_transforms['train']),
    'test_data': datasets.ImageFolder(test_dir, transform = data_transforms['test']),
    'valid_data': datasets.ImageFolder(valid_dir, transform = data_transforms['valid'])
}

# TODO: Using the image datasets and the trainforms, define the dataloaders
dataloaders = {
    'train_loader': torch.utils.data.DataLoader(image_datasets['train_data'], batch_size=64, shuffle=True),
    'test_loader': torch.utils.data.DataLoader(image_datasets['test_data'], batch_size=32, shuffle=True),
    'valid_loader' : torch.utils.data.DataLoader(image_datasets['valid_data'], batch_size=32, shuffle=True)
}

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [14]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

**Note for Workspace users:** If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.

In [17]:
# TODO: Build and train your network
# Load pretrained network
model = models.vgg16(pretrained=True)
# freeze parameter
for param in model.parameters():
    param.requires_grad=False
# Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
classifier = nn.Sequential(
    nn.Linear(25088, 4096),
    nn.ReLU(),
    nn.Dropout(p=0.05),
    nn.Linear(4096, 102),
    nn.LogSoftmax(dim=1)
)
model.classifier = classifier
# Train the classifier layers using backpropagation using the pre-trained network to get the features
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)
# check GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
epochs = 5
batchs = 0
running_loss = 0
print_every = 20
trainloader = dataloaders['train_loader']
validloader = dataloaders['valid_loader']
for epoch in range(epochs):
    for images, labels in trainloader:
        batchs += 1
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        logps = model.forward(images)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if batchs % print_every == 0:
#Track the loss and accuracy on the validation set to determine the best hyperparameters
            valid_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for images, labels in validloader:
                    images, labels = images.to(device), labels.to(device)
                    logps = model.forward(images)
                    loss = criterion(logps, labels)
                    valid_loss += loss.item()
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equality = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equality.type(torch.FloatTensor)).item()
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Validation loss: {valid_loss/len(validloader):.3f}.. "
                  f"Validation accuracy: {accuracy/len(validloader):.3f}")
            running_loss = 0
            model.train()

Epoch 1/5.. Train loss: 16.901.. Validation loss: 3.593.. Validation accuracy: 0.236
Epoch 1/5.. Train loss: 3.135.. Validation loss: 2.145.. Validation accuracy: 0.450
Epoch 1/5.. Train loss: 2.360.. Validation loss: 1.630.. Validation accuracy: 0.582
Epoch 1/5.. Train loss: 2.050.. Validation loss: 1.400.. Validation accuracy: 0.636
Epoch 1/5.. Train loss: 1.804.. Validation loss: 1.238.. Validation accuracy: 0.660
Epoch 2/5.. Train loss: 1.715.. Validation loss: 1.243.. Validation accuracy: 0.680
Epoch 2/5.. Train loss: 1.701.. Validation loss: 1.170.. Validation accuracy: 0.693
Epoch 2/5.. Train loss: 1.628.. Validation loss: 1.098.. Validation accuracy: 0.732
Epoch 2/5.. Train loss: 1.515.. Validation loss: 1.157.. Validation accuracy: 0.724
Epoch 2/5.. Train loss: 1.593.. Validation loss: 1.065.. Validation accuracy: 0.705
Epoch 3/5.. Train loss: 1.336.. Validation loss: 1.015.. Validation accuracy: 0.744
Epoch 3/5.. Train loss: 1.468.. Validation loss: 1.028.. Validation accurac

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [18]:
# TODO: Do validation on the test set
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
testloader = dataloaders['test_loader']
model.eval()
accuracy = 0
test_loss = 0
with torch.no_grad():
    for images, labels in testloader:
        # image and labels to gpu
        images, labels = images.to(device), labels.to(device)
        logps = model.forward(images)
        # calculate loss
        loss = criterion(logps, labels)
        test_loss += loss.item()
        # calculate accuracy
        ps = torch.exp(logps)
        top_p, top_class = ps.topk(1, dim=1)
        equality = top_class == labels.view(*top_class.shape)
        accuracy += torch.mean(equality.type(torch.FloatTensor)).item()
# Result
print(f"Test Loss: {test_loss/len(testloader):.3f}.. "
      f"Test Accuracy: {100*accuracy/len(testloader):.3f}%")

Test Loss: 1.004.. Test Accuracy: 75.709%


## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [20]:
# TODO: Save the checkpoint 
model.class_to_idx = image_datasets['train_data'].class_to_idx
torch.save({'input_size': 25088,
            'output_size': 102,
            'classifier': model.classifier,
            'state_dict': model.state_dict(),
            'class_to_idx': model.class_to_idx}, 'checkpoint.pth')

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [23]:
# TODO: Write a function that loads a checkpoint and rebuilds the model
def load_model(path):
    checkpoint = torch.load(path)
    model.class_to_idx = checkpoint['class_to_idx']
    model.load_state_dict(checkpoint['state_dict'])
    return model
model = load_model('checkpoint.pth')
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [52]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    # TODO: Process a PIL image for use in a PyTorch model
    pil_image = Image.open(image)
    pil_image = pil_image.resize((256,256))
    # crop the image
    width, height = pil_image.size
    left = (width - 224) / 2
    top = (height - 224) / 2
    right = left + 224
    bottom = top + 224
    pil_image = pil_image.crop((left, top, right, bottom))
    np_image = np.array(pil_image) / 255
    # normalize image
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    np_image = (np_image - means) / std
    np_image = np_image.transpose((2,0,1))
    return np_image

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [45]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [80]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # TODO: Implement the code to predict the class from an image file
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    np_image = process_image(image_path)
    #convert to tensor
    #image_tensor = torch.from_numpy(np_image).type(torch.FloatTensor)
    # add batch size 1
    #image = image_tensor.unsqueeze_(0)
    model.eval()
    with torch.no_grad():
        image_tensor = np_image.to(device)
        image_tensor.unsqueeze_(0)
        image_tensor.float()
        logps = model.forward(image_tensor)
        ps = torch.exp(logps)
        top_ps, top_class_idx = ps.topk(topk, dim=1)
    idx_to_class = {val: key for key, val in model.top_class_idx.items()}
    classes = [idx_to_class[index] for index in indices]
    probs = top_ps.cpu().numpy()[0]
    return probs, classes

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [81]:
# TODO: Display an image along with the top 5 classes
image_path = "flowers/test/10/image_07090.jpg"
probs, classes = predict(image_path, model)
print (probs)
print (classes)

AttributeError: 'numpy.ndarray' object has no attribute 'to'

In [ ]:
import torch
import argparse
import json
import numpy as np
from PIL import Image

# Define command line arguments
parser = argparse.ArgumentParser(description='Predict flower name from an image')
parser.add_argument('image_path', default='./flowers/test/24/image_06849.jpg',type=str,help='Path to image')
parser.add_argument('checkpoint', type=str,help='Model checkpoint path')
parser.add_argument('--top_k', type=int,help='Top K most likely classes', default=5)
parser.add_argument('--category_names', type=str, default='cat_to_name.json',help='Mapping of categories to real names')
parser.add_argument('--gpu', default= 'gpu', type=str, help='Use GPU')

# Parse command line arguments
args = parser.parse_args()

# Load category names
with open(args.category_names, 'r') as f:
    cat_to_name = json.load(f)

# Load checkpoint
checkpoint = torch.load(args.checkpoint)

# Build and load model
if checkpoint['arch'] == 'vgg13':
    model = models.vgg13(pretrained=True)
elif checkpoint['arch'] == 'vgg16':
    model = models.vgg16(pretrained=True)
else:
    print("Architecture not supported.")
    exit()

model.classifier = checkpoint['classifier']
model.load_state_dict(checkpoint['state_dict'])
model.class_to_idx = checkpoint['class_to_idx']

# Move model to GPU
if args.gpu:
    if torch.cuda.is_available():
        device = torch.device("cuda")
    else:
        print("GPU not available, using CPU instead.")
        device = torch.device("cpu")
else:
    device = torch.device("cpu")

model.to(device)

# Process image
def process_image(image):
    """ Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    """
    pil_image = Image.open(image).convert("RGB")
    pil_image = pil_image.resize((256, 256))
    pil_image = pil_image.crop((16, 16, 240, 240))
    np_image = np.array(pil_image)
    np_image = np_image/255
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    np_image = (np_image - mean) / std
    np_image = np_image.transpose((2, 0, 1))
    return np_image

image_path = args.image_path
image = process_image(image_path)
image = torch.from_numpy(image).type(torch.FloatTensor)
# batch dimension
image.unsqueeze_(0)
# tensor to GPU
if args.gpu:
    image = image.to(device)
model.eval()

# Disable gradient calculations
with torch.no_grad():
    output = model.forward(image)

# Calculate probabilities
probabilities = torch.exp(output)
top_probs, top_classes = probabilities.topk(args.top_k, dim=1)

# probabilities and classes to lists
top_probs = top_probs.cpu().numpy().tolist()[0]
top_classes = top_classes.cpu().numpy().tolist()[0]

# Convert class indices to class labels
idx_to_class = {value: key for key, value in model.class_to_idx.items()}
top_labels = [cat_to_name[idx_to_class[cls]] for cls in top_classes]

# Print top K classes and their probabilities
for i in range(args.top_k):
    print(f"{cat_to_name[str(top_classes[i])]}: {probs[top_classes[i]]*100:.2f}%")


In [ ]:
import argparse
import json
import os

import torch
from torch import nn, optim
from torchvision import datasets, transforms, models


def get_args():
    parser = argparse.ArgumentParser(description='Train a neural network on a given dataset')
    parser.add_argument('data_dir', help='path to dataset')
    parser.add_argument('--save_dir', default='checkpoint.pth', help='directory to save the checkpoint')
    parser.add_argument('--arch', default='vgg16', help='pretrained model architecture to use')
    parser.add_argument('--learning_rate', type=float, default=0.001, help='learning rate for optimizer')
    parser.add_argument('--hidden_units', type=int, default=512, help='number of hidden units in classifier')
    parser.add_argument('--epochs', type=int, default=10, help='number of epochs to train')
    parser.add_argument('--gpu', action='store_true', help='use GPU for training if available')
    return parser.parse_args()


def build_model(arch, hidden_units):
    # Load pretrained model
    model = getattr(models, arch)(pretrained=True)

    # Freeze parameters of pretrained model
    for param in model.parameters():
        param.requires_grad = False

    # Replace classifier of pretrained model
    classifier = nn.Sequential(
        nn.Linear(model.classifier[0].in_features, hidden_units),
        nn.ReLU(),
        nn.Dropout(p=0.5),
        nn.Linear(hidden_units, 102),
        nn.LogSoftmax(dim=1)
    )
    model.classifier = classifier

    return model


def train(model, dataloaders, criterion, optimizer, device):
    for epoch in range(args.epochs):
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

    return model


def save_checkpoint(model, optimizer, args):
    checkpoint = {
        'arch': args.arch,
        'classifier': model.classifier,
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'class_to_idx': model.class_to_idx
    }
    torch.save(checkpoint, args.save_dir)


def main():
    args = get_args()

    # Define transforms for the training, validation, and testing sets
    data_transforms = {
        'train': transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(30),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'valid': transforms.Compose([
           


In [ ]:
# TODO: Display an image along with the top 5 classes
image_path = "flowers/test/23/image_03440.jpg"

probs, classes = predict(image_path, model)
# class to names
class_names = [cat_to_name[str(cls_idx)] for cls_idx in classes]
# Load img
image = Image.open(image_path)

# subplots
fig, (ax1, ax2) = plt.subplots(figsize=(6,9), ncols=1, nrows=2)

# Show image
ax1.imshow(image)
ax1.axis('off')
ax1.set_title(cat_to_name[str(image_path.split('/')[2])])

# bar chart
y_pos = np.arange(len(class_names))
ax2.barh(y_pos, probs, align='center')
ax2.set_yticks(y_pos)
ax2.set_yticklabels(class_names)
ax2.invert_yaxis() 
plt.show()